In [59]:
from google.colab import drive
drive.mount('/content/drive' , force_remount=True)


Mounted at /content/drive


In [60]:
!pip install torchinfo

### Preprocess Climate Data

In [ ]:
import pandas as pd

file_path = '/content/drive/MyDrive/NNProject/cleaned_climate_pca_extensive.csv'

climate_df = pd.read_csv(file_path)
climate_df['date'] = climate_df['date'].str.slice(0, 10)
climate_df['date'] = pd.to_datetime(climate_df['date'], format="%Y-%m-%d", errors="coerce")
climate_df.index = climate_df['date']

In [ ]:
climate_df

In [ ]:

from sklearn.preprocessing import MinMaxScaler

numeric_cols = climate_df.select_dtypes(include=['float64', 'int64']).columns.to_list()
numeric_cols.remove('lat')
numeric_cols.remove('lon')
print(numeric_cols)
non_numeric_cols = climate_df.select_dtypes(exclude=['float64', 'int64']).columns.to_list() + ['lat', 'lon']
print(non_numeric_cols)
climate_df[numeric_cols] = climate_df[numeric_cols].astype(float)
climate_df[non_numeric_cols] = climate_df[non_numeric_cols]
scaler = MinMaxScaler()
scaled_df = pd.DataFrame(scaler.fit_transform(climate_df[numeric_cols]), columns=numeric_cols, index=climate_df.index)

climate_data = pd.concat([scaled_df, climate_df[non_numeric_cols]], axis=1)


In [ ]:
climate_data

In [ ]:

print(climate_data["lat"].unique())
print(climate_data["lon"].unique())

### Preprocess Energy Data

In [ ]:
file_path = '/content/drive/MyDrive/NNProject/daily_energy.csv'

energy_df = pd.read_csv(file_path)
energy_df['Dates'] = energy_df['Dates'].str.slice(0, 10)
energy_df['Dates'] = pd.to_datetime(energy_df['Dates'], format="%Y-%m-%d", errors="coerce")
energy_df.index = energy_df['Dates']



In [ ]:

numeric_cols = energy_df.select_dtypes(include=['float64', 'int64']).columns.to_list()
numeric_cols.remove('latitude')
numeric_cols.remove('longitude')
non_numeric_cols = energy_df.select_dtypes(exclude=['float64', 'int64']).columns.to_list() + ['latitude', 'longitude']
energy_df[numeric_cols] = energy_df[numeric_cols].astype(float)

scaler = MinMaxScaler()
scaled_df = pd.DataFrame(scaler.fit_transform(energy_df[numeric_cols]), columns=numeric_cols, index=energy_df.index)
energy_data = pd.concat([scaled_df, energy_df[non_numeric_cols]], axis=1)

### State-Lat-Lon Map

In [ ]:

import numpy as np

lat_range = np.array([10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32])
lon_range = np.array([72, 74, 76, 78, 80, 82, 84, 86])

energy_data["latitude"] = energy_data["latitude"].apply(lambda x: lat_range[np.argmin(np.abs(lat_range - x))])
energy_data["longitude"] = energy_data["longitude"].apply(lambda x: lon_range[np.argmin(np.abs(lon_range - x))])

## PreTrain on Climate Data

### Time-Series Sequences

In [ ]:
import numpy as np

def lagged_climate_input(data, seq_length=28):
    X = []
    for _, group in data.groupby(["lat", "lon"]):
        group = group.sort_index()
        features = group.drop(columns=["date", "lat", "lon"])
        for row in range(len(features) - seq_length):
            X.append(features.iloc[row:row+seq_length].values)
    return np.array(X)

climate_X = lagged_climate_input(climate_data, 12)


In [ ]:
climate_X.shape

### Model

In [ ]:
import torch
import torch.nn as nn
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class ClimatePreTrainer(nn.Module):
    def __init__(self, input_size, hidden_size=64, latent_size=32, num_layers=1, dropout=0.3, aug_num_segments = 2):
        super(ClimatePreTrainer, self).__init__()

        self.encoder = nn.LSTM(input_size, hidden_size, num_layers=num_layers,
                               batch_first=True, bidirectional=True, dropout=dropout)

        self.encoder_bn = nn.BatchNorm1d(hidden_size * 2)

        self.to_latent = nn.Linear(hidden_size * 2, latent_size)
        self.from_latent = nn.Linear(latent_size, hidden_size * 2)

        self.decoder = nn.LSTM(hidden_size * 2, hidden_size, num_layers=num_layers,
                               batch_first=True, bidirectional=True, dropout=dropout)

        self.decoder_bn = nn.BatchNorm1d(hidden_size * 2)

        self.output_layer = nn.Linear(hidden_size * 2, input_size)
        self.dropout = nn.Dropout(dropout)
        self.aug_num_segments = aug_num_segments

    def forward(self, x, augment = False):
        if augment:
            x = self.augment_data(x)

        encoded_x, _ = self.encoder(x)
        encoded_x = self.dropout(encoded_x)

        batch_size, seq_len, feat_dim = encoded_x.shape
        norm_encoded = self.encoder_bn(encoded_x.contiguous().view(-1, feat_dim))
        norm_encoded = norm_encoded.view(batch_size, seq_len, feat_dim)

        latent = self.to_latent(encoded_x)
        expanded = self.from_latent(latent)

        decoded_x, _ = self.decoder(expanded)
        decoded_x = self.dropout(decoded_x)

        batch_size, seq_len, feat_dim = decoded_x.shape
        norm_decoded = self.decoder_bn(decoded_x.contiguous().view(-1, feat_dim))
        norm_decoded = norm_decoded.view(batch_size, seq_len, feat_dim)

        reconstructed_x = self.output_layer(decoded_x)
        return reconstructed_x, latent

    def augment_data(self, x):
        """Applies data augmentation techniques to the input data."""
        # 1. Jittering: Add random noise to the input
        noise_factor = 0.05  # Adjust the noise level as needed
        noise = torch.randn_like(x) * noise_factor
        x = x + noise

        # 2. Scaling: Scale the input by a random factor
        scale_factor = np.random.uniform(0.9, 1.1)  # Adjust the scaling range as needed
        x = x * scale_factor

        # 3. Permutation: Randomly permute segments within the time series
        # (You might need to adapt this based on your specific data structure)
        # Adjust the number of segments as needed
        segment_length = x.shape[1] // self.aug_num_segments
        indices = torch.randperm(self.aug_num_segments)
        permuted_x = torch.cat([x[:, i * segment_length:(i + 1) * segment_length] for i in indices], dim=1)
        x = permuted_x

        return x


In [ ]:
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from torchinfo import summary

num_epochs = 50
batch_size = 64
lr = 0.01

patience = 5
best_val_loss = float('inf')
counter = 0

climate_X_tensor = torch.tensor(climate_X, dtype=torch.float32)

train_X, val_X = train_test_split(climate_X_tensor, test_size=0.2, random_state=42)

train_dataset = TensorDataset(train_X)
val_dataset = TensorDataset(val_X)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
climate_pretrained_model = ClimatePreTrainer(input_size=climate_X.shape[2], num_layers=2, aug_num_segments=4).to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(climate_pretrained_model.parameters(), lr=lr, weight_decay=1e-5)

for epoch in range(num_epochs):
    climate_pretrained_model.train()
    train_loss = 0
    for batch in train_loader:
        batch = batch[0].to(device)
        optimizer.zero_grad()

        reconstructed_x, _ = climate_pretrained_model(batch, augment=True)
        loss = criterion(reconstructed_x, batch)

        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    avg_train_loss = train_loss / len(train_loader)

    climate_pretrained_model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            batch = batch[0].to(device)
            reconstructed_x, _ = climate_pretrained_model(batch)
            loss = criterion(reconstructed_x, batch)
            val_loss += loss.item()
    avg_val_loss = val_loss / len(val_loader)

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.6f}, Val Loss: {avg_val_loss:.6f}")

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        counter = 0
        torch.save(climate_pretrained_model.state_dict(), 'climate_pretrained_model.pth')
        print(f"Validation loss improved. Saving model.")
    else:
        counter += 1
        print(f"Validation loss did not improve. Patience Counter: {counter}/{patience}")
        if counter >= patience:
            print("Early stopping triggered.")
            break


#### Freeze Encoder Model

In [ ]:
climate_pretrained_model = ClimatePreTrainer(input_size=climate_X.shape[2], num_layers=2).to(device)
climate_pretrained_model.load_state_dict(torch.load('climate_pretrained_model.pth'))

for param in climate_pretrained_model.parameters():
    param.requires_grad = False

## FineTune Downstream

In [ ]:
climate_data = climate_data.reset_index(drop=True)
energy_data = energy_data.reset_index(drop=True)

### Merge DataSources

In [ ]:
energy_data = energy_data.rename(columns={"latitude": "lat", "longitude": "lon"})

In [ ]:
merged_df = pd.merge(
    energy_data,
    climate_data,
    how="left",
    left_on=["lat", "lon", "Dates"],
    right_on=["lat", "lon", "date"],
    suffixes=("", "_climate"),
)

In [ ]:
merged_df.shape

In [ ]:

merged_df

In [ ]:
merged_df = merged_df.drop(columns=["Dates"])
merged_df = merged_df.dropna().reset_index(drop=True)
merged_df.shape

In [ ]:
merged_df.columns

In [ ]:
from sklearn.preprocessing import LabelEncoder
categorical_cols = merged_df.select_dtypes(include=["object"]).columns
for col in categorical_cols:
    le = LabelEncoder()
    merged_df[col] = le.fit_transform(merged_df[col])


In [ ]:

def lagged_merged_data(data, seq_length=28):
    X = []
    y = []
    for _, group in data.groupby(["lat", "lon"]):
        group = group.sort_values("date")
        features = group.drop(columns=["date", "lat", "lon", "States", "Usage"])
        target = group["Usage"]
        for row in range(len(features) - seq_length):
            X.append(features.iloc[row:row+seq_length].values)
            y.append(target.iloc[row+seq_length])
    return np.array(X), np.array(y)

merged_X, targt_y = lagged_merged_data(merged_df, 12)

In [ ]:

merged_X.shape

In [ ]:

class EnergyPrediction(nn.Module):
    def __init__(self, encoder, input_size, hidden_size=64, aug_num_segments=4):
        super(EnergyPrediction, self).__init__()
        self.encoder = encoder.encoder
        self.regressor = nn.Sequential(
            nn.Linear(hidden_size * 2, hidden_size),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1)
        )
        self.aug_num_segments = aug_num_segments

    def forward(self, x, augment=False):
        if augment:
            x = self.augment_data(x)
        encoded_x, _ = self.encoder(x)  # [batch, seq_len, hidden*2]
        x = self.regressor(encoded_x[:, -1, :])  # Use last timestep
        return x  # [batch, 1]

    def augment_data(self, x):
        """Applies data augmentation techniques to the input data."""
        # 1. Jittering: Add random noise to the input
        noise_factor = 0.05  # Adjust the noise level as needed
        noise = torch.randn_like(x) * noise_factor
        x = x + noise

        # 2. Scaling: Scale the input by a random factor
        scale_factor = np.random.uniform(0.9, 1.1)  # Adjust the scaling range as needed
        x = x * scale_factor

        # 3. Permutation: Randomly permute segments within the time series
        # Adjust the number of segments as needed
        segment_length = x.shape[1] // self.aug_num_segments
        indices = torch.randperm(self.aug_num_segments)
        permuted_x = torch.cat([x[:, i * segment_length:(i + 1) * segment_length] for i in indices], dim=1)
        x = permuted_x

        return x


In [ ]:
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from torchinfo import summary

batch_size = 32

merged_X_tensor = torch.tensor(merged_X, dtype=torch.float32)
targt_y_tensor = torch.tensor(targt_y, dtype=torch.float32)
merged_X_tensor = merged_X_tensor.view(merged_X_tensor.shape[0], merged_X_tensor.shape[1], -1)
targt_y_tensor = targt_y_tensor.view(targt_y_tensor.shape[0], 1, -1).view(-1, 1, 1).repeat(1, merged_X_tensor.shape[1], 1)
merged_tensor = torch.cat((merged_X_tensor, targt_y_tensor), dim=2)

train, val = train_test_split(merged_tensor, test_size=0.2, random_state=42)
train_X = train[:, :, :-1]
train_y = train[:, :, -1]
val_X = val[:, :, :-1]
val_y = val[:, :, -1]

train_X = train_X.view(train_X.shape[0], train_X.shape[1], -1)
val_X = val_X.view(val_X.shape[0], val_X.shape[1], -1)
train_loader = DataLoader(TensorDataset(train_X, train_y), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(TensorDataset(val_X, val_y), batch_size=batch_size, shuffle=False)


climate_pretrained_model.eval()
energy_model = EnergyPrediction(climate_pretrained_model, input_size=merged_X.shape[2], aug_num_segments=4).to(device)
print("Model Summary:")
print(energy_model)
print(summary(energy_model, input_size=(1, 14, merged_X.shape[2])))

In [ ]:
lr = 0.001
num_epochs = 50

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(energy_model.parameters(), lr=lr)
energy_model.train()
loss_vector = []
for epoch in range(num_epochs):
    for batch in train_loader:
        inputs, targets = batch[0].to(device), batch[1].to(device)
        optimizer.zero_grad()
        outputs = energy_model(inputs, augment=True)
        loss = criterion(outputs, targets)
        loss_vector.append(loss.item())
        loss.backward()
        optimizer.step()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

In [ ]:
from matplotlib import pyplot as plt
plt.plot(loss_vector)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('Training Loss With Frozen Pretrained Model')
plt.show()

In [ ]:

for param in energy_model.encoder.parameters():
    param.requires_grad = True

loss_vector = []
for epoch in range(num_epochs):
    for batch in train_loader:
        inputs, targets = batch[0].to(device), batch[1].to(device)
        optimizer.zero_grad()
        outputs = energy_model(inputs)
        loss = criterion(outputs, targets)
        loss_vector.append(loss.item())
        loss.backward()
        optimizer.step()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

torch.save(energy_model.state_dict(), 'energy_model.pth')

In [ ]:
plt.plot(loss_vector)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('Training Loss With Fine-tuning Model')
plt.show()

#### Model Analysis

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

energy_model.eval()
val_predictions = []
val_targets = []
with torch.no_grad():
    for batch in val_loader:
        inputs, targets = batch[0].to(device), batch[1].to(device)
        outputs = energy_model(inputs)
        val_predictions.append(outputs.cpu().numpy())
        val_targets.append(targets.cpu().numpy())
val_predictions = np.concatenate(val_predictions)
val_targets = np.concatenate(val_targets)
val_predictions = scaler.inverse_transform(val_predictions)
val_targets = scaler.inverse_transform(val_targets)
val_targets = val_targets[:, [0]]
mse = mean_squared_error(val_targets, val_predictions)
mae = mean_absolute_error(val_targets, val_predictions)
r2 = r2_score(val_targets, val_predictions)
print(f"Validation MSE: {mse:.4f}")
print(f"Validation MAE: {mae:.4f}")
print(f"Validation R2: {r2:.4f}")

plt.figure(figsize=(10, 5))
plt.plot(val_targets, label='True Values')
plt.plot(val_predictions, label='Predicted Values')
plt.xlabel('Sample Index')
plt.ylabel('Energy Consumption')
plt.title('True vs Predicted Energy Consumption')
plt.legend()
plt.show()


In [ ]:
def predict(state, date, model_path="energy_model.pth", meta_path="energy_model.pkl", true_data_path="data/daily_energy.csv", lagged_days=14):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = EnergyPrediction(climate_pretrained_model, input_size=merged_X.shape[2]).to(device)
    model.load_state_dict(torch.load(model_path))
    model.eval()

    true_data = pd.read_csv(true_data_path)
    true_data = true_data[true_data['States'] == state]
    true_data['Dates'] = true_data['Dates'].str.slice(0, 10)
    true_data['Dates'] = pd.to_datetime(true_data['Dates'], format="%Y-%m-%d", errors="coerce")
    true_data = true_data.sort_values("Dates")

    end_idx = true_data[true_data['Dates'] == date].index[0]
    start_idx = end_idx - lagged_days + 1
    if start_idx < 0:
        raise ValueError("Not enough data to predict.")
    past_window = true_data.iloc[start_idx:end_idx + 1]
    features = past_window.drop(columns=["Dates", "States", "Usage"])
    input_tensor = torch.tensor(features.values, dtype=torch.float32).unsqueeze(0).to(device)

    with torch.no_grad():
        prediction = model(input_tensor)
        prediction = prediction.cpu().numpy()[0][0]

    actual = true_data.iloc[end_idx]['Usage']

    return prediction, actual


In [ ]:
prediction, actual = predict("Maharashtra", "06/01/2019")
print(f"Predicted Usage: {prediction}")
print(f"Actual Usage: {actual}")